In [1]:
# import libraries
import uproot
import numpy as np
import matplotlib.pyplot as plt

import Style as style

from scipy.optimize import curve_fit
from scipy.stats import norm

save_fig = False

## **I. Initial Checks**

### **1. Injected Anti-Sexaquarks**

In [18]:
file = uproot.open("/home/ceres/borquez/some/output/A1.8_fix/AnalysisResults_merged.root")
# read the tree "Injected" that's inside the TList "Trees"
tree = file["Trees"][0]

In [ ]:
# convert branch "SM" into a numpy array
SM = tree["Sexa_M_ini"].array() # mass
SPx = tree["Sexa_Px_ini"].array() # momentum
SPy = tree["Sexa_Py_ini"].array()
SPz = tree["Sexa_Pz_ini"].array()

SPt = np.sqrt(SPx**2 + SPy**2)
SPhi = np.arctan2(SPy, SPx)
SPhi = np.where(SPhi < 0, SPhi + 2 * np.pi, SPhi)
SMt = np.sqrt(SM**2 + SPt**2)
SRapidity = np.arcsinh(SPz / SMt)

variables = [SPt, SPhi, SRapidity]
variable_names = [r"$p_T$ (GeV/c)", r"$\phi$ (rad)", "Rapidity"]

plt.figure(figsize=(6 * len(variables), 6))

for i in range(len(variables)):

    ax = plt.subplot(1, len(variables), i + 1)

    n, bins, patches = ax.hist(variables[i], bins=100, color=style.colors[i], histtype="step", label=r"MC Gen. Injected $\bar{S}$", linewidth=2)

    n_entries = len(variables[i])
    stats = f"Entries: {n_entries:.0f}"
    ax.hist(variables[i], bins=100, alpha=0., label=stats) # invisible histogram to show stats on the legend

    ax.set_ylim(top=1.2*np.max(n))
    ax.set_ylabel("Counts", loc='top', fontsize=12, fontweight='bold')
    ax.set_xlabel(variable_names[i], loc='right', fontsize=12)
    ax.tick_params(direction='in', top=True, right=True)
    ax.grid(True, linestyle='--', alpha=0.5)

    plt.legend(fontsize=12)

plt.tight_layout()
if (save_fig): plt.savefig("QA_InjectedSexaquarks.svg")
plt.show()

### **2. Struck Nucleon**

In [ ]:
# plot p, phi, theta, px, py, pz of struck nucleons
def gaussian(x, a, mu, sigma):
    return a * np.exp(-0.5 * ((x - mu) / sigma)**2)

file = uproot.open("../macros/InjectedResults.root")
tree = file["IC"]

# convert branch "SM" into a numpy array
NPx = tree["NPx"].array() # momentum
NPy = tree["NPy"].array()
NPz = tree["NPz"].array()

NP = np.sqrt(NPx**2 + NPy**2 + NPz**2)
NPt = np.sqrt(NPx**2 + NPy**2)
NPhi = np.arctan2(NPy, NPx)
NPhi = np.where(NPhi < 0, NPhi + 2 * np.pi, NPhi)
NTheta = np.arccos(NPz / NP)

variables = [NP, NPhi, NTheta, NPx, NPy, NPz]
variable_names = [r"$p$ (GeV/c)", r"$\phi$ (rad)", r"$\theta$ (rad)", r"$p_x$ (GeV/c)", r"$p_y$ (GeV/c)", r"$p_z$ (GeV/c)"]

n = len(variables)
rows = n // 3 + (n % 3 > 0)  # Calculate the number of rows needed
cols = 3  # Set the number of columns

plt.figure(figsize=(6 * cols, 6 * rows))

for i in range(n):

    row = i // cols
    col = i % cols
    ax = plt.subplot(rows, cols, row * cols + col + 1)

    bin_content, bins, patches = ax.hist(variables[i], bins=100, color=style.colors[i], histtype="step", label=r"MC Gen. Struck Nucleon", linewidth=2)

    n_entries = len(variables[i])
    stats = f"Entries: {n_entries:.0f}"
    ax.hist(variables[i], bins=100, alpha=0., label=stats) # invisible histogram to show stats on the legend

    # if the var is momentum, fit a gaussian to it
    if variable_names[i] == r"$p$ (GeV/c)":
        avg = np.mean(variables[i])
        std = np.std(variables[i])
        popt, _ = curve_fit(gaussian, bins[:-1], bin_content, p0=[1, avg, std])
        x = np.linspace(np.min(variables[i]), np.max(variables[i]), 1000)
        ax.plot(x, gaussian(x, *popt), color=style.colors[3], linestyle="-", label=r"Gaussian: $\mu$ = %.3f, $\sigma$ = %.3f" % (popt[1], popt[2]), linewidth=2)

    ax.set_ylim(top=1.3 * np.max(bin_content))
    ax.set_ylabel("Counts", loc='top', fontsize=12, fontweight='bold')
    ax.set_xlabel(variable_names[i], loc='right', fontsize=12)
    ax.tick_params(direction='in', top=True, right=True)
    ax.grid(True, linestyle='--', alpha=0.5)

    plt.legend(fontsize=12)

plt.tight_layout()
if (save_fig): plt.savefig("QA_StruckNucleons.svg")
plt.show()

### **3. Anti-Sexaquark—Nucleon Interaction**

In [ ]:
# plot radius of signal interactions
file = uproot.open("../sexaquark/output_sig_parallel/AnalysisResults_testQAplots.root")

histograms = file["Hists/"]
hist_dict = {hist.name: hist for hist in histograms}

plt.figure(figsize=(6, 6))

hist_name = "MCGen_All_AntiSexaquark_Radius"
values, edges = hist_dict[hist_name].to_numpy()

plot_name = "MC Gen. Signal Interaction"
plt.hist(edges[:-1], edges, weights=values, label=plot_name, histtype='step', color=style.colors[7], linewidth=2)

n_entries = np.sum(values)
stats = f"Entries: {n_entries:.0f}"
plt.hist(edges[:-1], edges, alpha=0., label=stats) # invisible histogram to show stats on the legend

plt.ylabel('Counts', loc='top', fontsize=12, fontweight='bold')
plt.xlabel('Radius (cm)', loc='right', fontsize=12)
plt.ylim(top=1.3 * np.max(values))
plt.tick_params(direction='in', top=True, right=True)
plt.grid(True, linestyle='--', alpha=0.5)

# add vertical lines on 5 and 180
plt.axvline(x=5, color=style.colors[3], alpha=0.8, linestyle="--", linewidth=2)
plt.axvline(x=180, color=style.colors[3], alpha=0.8, linestyle="--", linewidth=2)

plt.legend(loc="upper center", fontsize=12)

plt.tight_layout()
if (save_fig): plt.savefig("QA_SignalInteraction_Radius.svg")
plt.show()

In [ ]:
# plot pt, phi and rapidity of the injected anti-sexaquarks (after interaction)

variables = ["Mass", "Pt", "Pz", "Rapidity"] # TEMPORARY: change Pz by Phi!
variable_names = [r"Mass (GeV/$c^2$)", r"$p_T$ (GeV/c)", r"$\phi$ (rad)", "Rapidity"]

n = len(variables)
cols = 2
rows = 2
plt.figure(figsize=(6 * cols, 6 * rows))

for i in range(len(variables)):

    row = i // cols
    col = i % cols
    plt.subplot(rows, cols, row * cols + col + 1)

    hist_name = f"MCGen_All_AntiSexaquark_{variables[i]}"
    values, edges = hist_dict[hist_name].to_numpy()

    plot_name = r"MC Gen. $\bar{S}$ (after interaction)"
    plt.hist(edges[:-1], edges, weights=values, label=plot_name, histtype='step', color=colors[i], linewidth=2)

    n_entries = np.sum(values)
    stats = f"Entries: {n_entries:.0f}"
    plt.hist(edges[:-1], edges, weights=values, alpha=0., label=stats) # invisible histogram to show stats on the legend

    plt.ylim(top=1.2*np.max(values))
    plt.ylabel("Counts", loc='top', fontsize=12, fontweight='bold')
    plt.xlabel(variable_names[i], loc='right', fontsize=12)
    plt.tick_params(direction='in', top=True, right=True)
    plt.grid(True, linestyle='--', alpha=0.5)

    plt.legend(loc="upper left", fontsize=12)

plt.tight_layout()
if (save_fig): plt.savefig("QA_SexaquarksAfterInteraction.svg")
plt.show()

...

In [ ]:
# load files
file_bkg = uproot.open("../sexaquark/output_bkg_parallel/bkg.root")
histograms_bkg = file_bkg["Hists/"]
hist_dict_bkg = {hist.name: hist for hist in histograms_bkg}

file_sig = uproot.open("../sexaquark/output_sig_parallel/signal.root")
histograms_sig = file_sig["Hists/"]
hist_dict_sig = {hist.name: hist for hist in histograms_sig}

## **II. MC Generated**

In [ ]:
# plot z vertex of the collisions
plt.figure(figsize=(6, 6))

hist_names = ["QA_MCGen_VertexX", "QA_MCGen_VertexY", "QA_MCGen_VertexZ"]
x_axis_names = ['MC Gen. x-vertex [cm]', 'MC Gen. y-vertex [cm]', 'MC Gen. z-vertex [cm]']

plt.figure(figsize=(len(hist_names) * 6, 6))

for i in range(len(hist_names)):

    plt.subplot(1, len(hist_names), i + 1)

    hist_name = hist_names[i]

    values_bkg, edges_bkg = hist_dict_bkg[hist_name].to_numpy()
    centers_bkg = (edges_bkg[:-1] + edges_bkg[1:]) / 2
    weights_bkg = values_bkg / np.max(values_bkg)
    bin_width_bkg = edges_bkg[1] - edges_bkg[0]
    plt.errorbar(centers_bkg, weights_bkg, xerr=bin_width_bkg/2, fmt='o', label="LHC20e3a", color=style.colors[0], markersize=4)

    values_sig, edges_sig = hist_dict_sig[hist_name].to_numpy()
    centers_sig = (edges_sig[:-1] + edges_sig[1:]) / 2
    weights_sig = values_sig / np.max(values_sig)
    bin_width_sig = edges_sig[1] - edges_sig[0]
    plt.errorbar(centers_sig, weights_sig, xerr=bin_width_sig/2, fmt='o', label="LHC23l1a3", color=style.colors[1], markersize=4)

    # plt.ylim(top=10* np.max([np.max(values_bkg), np.max(values_sig)]), bottom=1e-2)
    # plt.yscale('log')
    plt.ylabel('Normalized Counts', loc='top', fontsize=12, fontweight='bold')

    plt.xlabel(x_axis_names[i], loc='right', fontsize=12)

    plt.minorticks_on()
    plt.tick_params(direction='in', axis='both', which='both', top=True, right=True)
    plt.tick_params(which='minor', length=3)
    plt.tick_params(which='major', length=6)

    plt.legend(loc="upper left", fontsize=12, borderaxespad=1.5)

plt.tight_layout()
if (save_fig): plt.savefig("QA_MCGen_PVCoordinates.svg")
plt.show()

## **III. MC Reconstructed**

In [ ]:
# plot z vertex of the collisions
plt.figure(figsize=(6, 6))

hist_names = ["QA_VertexX", "QA_VertexY", "QA_VertexZ"]
x_axis_names = ['x-vertex [cm]', 'y-vertex [cm]', 'z-vertex [cm]']

plt.figure(figsize=(len(hist_names) * 6, 6))

for i in range(len(hist_names)):

    plt.subplot(1, len(hist_names), i + 1)

    hist_name = hist_names[i]

    values_bkg, edges_bkg = hist_dict_bkg[hist_name].to_numpy()
    centers_bkg = (edges_bkg[:-1] + edges_bkg[1:]) / 2
    weights_bkg = values_bkg # / np.max(values_bkg)
    bin_width_bkg = edges_bkg[1] - edges_bkg[0]
    # plt.errorbar(centers_bkg, weights_bkg, xerr=bin_width_bkg/2, fmt='o', label="LHC20e3a", color=style.colors[0], markersize=4)
    plt.hist(edges_bkg[:-1], edges_bkg, weights=weights_bkg, histtype='bar', label="LHC20e3a",  edgecolor=style.colors[0], linewidth=2)

    values_sig, edges_sig = hist_dict_sig[hist_name].to_numpy()
    centers_sig = (edges_sig[:-1] + edges_sig[1:]) / 2
    weights_sig = values_sig # / np.max(values_sig)
    bin_width_sig = edges_sig[1] - edges_sig[0]
    # plt.errorbar(centers_sig, weights_sig, xerr=bin_width_sig/2, fmt='o', label="LHC23l1a3", color=style.colors[1], markersize=4)
    plt.hist(edges_sig[:-1], edges_sig, weights=weights_sig, histtype='bar', label="LHC23l1a3",  edgecolor=style.colors[1], linewidth=2)

    # plt.ylim(top=10* np.max([np.max(values_bkg), np.max(values_sig)]), bottom=10e-1 * np.min([np.min(values_bkg), np.min(values_sig)]))
    # plt.yscale('log')
    plt.ylabel('Normalized Counts', loc='top', fontsize=12, fontweight='bold')

    plt.xlabel(x_axis_names[i], loc='right', fontsize=12)

    plt.minorticks_on()
    plt.tick_params(direction='in', axis='both', which='both', top=True, right=True)
    plt.tick_params(which='minor', length=3)
    plt.tick_params(which='major', length=6)

    plt.legend(loc="upper left", fontsize=12, borderaxespad=1.5)

plt.tight_layout()
if (save_fig): plt.savefig("QA_MCRec_PVCoordinates.svg")
plt.show()

In [ ]:
# plot DCAxy and DCAz of all tracks
hist_names = ["QA_Tracks_DCAxy", "QA_Tracks_DCAz"]

plt.figure(figsize=(len(hist_names) * 6, 6))

for i in range(len(hist_names)):

    plt.subplot(1, len(hist_names), i + 1)

    hist_name = hist_names[i]

    values_bkg, edges_bkg = hist_dict_bkg[hist_name].to_numpy()
    centers_bkg = (edges_bkg[:-1] + edges_bkg[1:]) / 2
    weights_bkg = values_bkg / sum(values_bkg) # np.max(values_bkg)
    bin_width_bkg = edges_bkg[1] - edges_bkg[0]
    plt.errorbar(centers_bkg, weights_bkg, xerr=bin_width_bkg/2, fmt='o', label="LHC20e3a", color=style.colors[0], markersize=4)

    values_sig, edges_sig = hist_dict_sig[hist_name].to_numpy()
    centers_sig = (edges_sig[:-1] + edges_sig[1:]) / 2
    weights_sig = values_sig / sum(values_sig) # np.max(values_sig)
    bin_width_sig = edges_sig[1] - edges_sig[0]
    plt.errorbar(centers_sig, weights_sig, xerr=bin_width_sig/2, fmt='o', label="LHC23l1a3", color=style.colors[1], markersize=4)

    plt.yscale('log')
    plt.ylabel('Normalized Counts', loc='top', fontsize=12, fontweight='bold')

    plt.xlabel(hist_name, loc='right', fontsize=12)

    plt.minorticks_on()
    plt.tick_params(direction='in', axis='both', which='both', top=True, right=True)
    plt.tick_params(which='minor', length=3)
    plt.tick_params(which='major', length=6)

    plt.legend(loc="upper left", fontsize=12, borderaxespad=1.5)

plt.tight_layout()
if (save_fig): plt.savefig("QA_Tracks_DCA.svg")
plt.show()

In [ ]:
# plot number of SPD tracklets
plt.figure(figsize=(6, 6))

hist_name = "QA_SPD_NTracklets"

values_bkg, edges_bkg = hist_dict_bkg[hist_name].to_numpy()
centers_bkg = (edges_bkg[:-1] + edges_bkg[1:]) / 2
weights_bkg = values_bkg / sum(values_bkg)
bin_width_bkg = edges_bkg[1] - edges_bkg[0]
plt.errorbar(centers_bkg, weights_bkg, xerr=bin_width_bkg/2, fmt='o', label="LHC20e3a", color=style.colors[0], markersize=4)

values_sig, edges_sig = hist_dict_sig[hist_name].to_numpy()
centers_sig = (edges_sig[:-1] + edges_sig[1:]) / 2
weights_sig = values_sig / sum(values_sig)
bin_width_sig = edges_sig[1] - edges_sig[0]
plt.errorbar(centers_sig, weights_sig, xerr=bin_width_sig/2, fmt='o', label="LHC23l1a3", color=style.colors[1], markersize=4)

plt.yscale('log')
plt.ylabel('Normalized Counts', loc='top', fontsize=12, fontweight='bold')

plt.xlabel('#SPD tracklets', loc='right', fontsize=12)

plt.minorticks_on()
plt.tick_params(direction='in', axis='both', which='both', top=True, right=True)
plt.tick_params(which='minor', length=3)
plt.tick_params(which='major', length=6)

plt.legend(loc="upper right", fontsize=12, borderaxespad=1.5)

plt.tight_layout()
if (save_fig): plt.savefig("QA_SPD_NTracklets.svg")
plt.show()

In [ ]:
# plot n tracks, n selected tracks, pt
hist_names = ["QA_NTracks", "QA_NSelectedTracks", "QA_Tracks_Pt"]

plt.figure(figsize=(len(hist_names) * 6, 6))

for i in range(len(hist_names)):

    plt.subplot(1, len(hist_names), i + 1)

    hist_name = hist_names[i]

    values_bkg, edges_bkg = hist_dict_bkg[hist_name].to_numpy()
    centers_bkg = (edges_bkg[:-1] + edges_bkg[1:]) / 2
    weights_bkg = values_bkg / sum(values_bkg) # np.max(values_bkg)
    bin_width_bkg = edges_bkg[1] - edges_bkg[0]
    plt.errorbar(centers_bkg, weights_bkg, xerr=bin_width_bkg/2, fmt='o', label="LHC20e3a", color=style.colors[0], markersize=4)

    values_sig, edges_sig = hist_dict_sig[hist_name].to_numpy()
    centers_sig = (edges_sig[:-1] + edges_sig[1:]) / 2
    weights_sig = values_sig / sum(values_sig) # np.max(values_sig)
    bin_width_sig = edges_sig[1] - edges_sig[0]
    plt.errorbar(centers_sig, weights_sig, xerr=bin_width_sig/2, fmt='o', label="LHC23l1a3", color=style.colors[1], markersize=4)

    plt.yscale('log')
    plt.ylabel('Normalized Counts', loc='top', fontsize=12, fontweight='bold')

    plt.xlabel(hist_name, loc='right', fontsize=12)

    plt.minorticks_on()
    plt.tick_params(direction='in', axis='both', which='both', top=True, right=True)
    plt.tick_params(which='minor', length=3)
    plt.tick_params(which='major', length=6)

    plt.legend(loc="upper right", fontsize=12, borderaxespad=1.5)

plt.tight_layout()
if (save_fig): plt.savefig("QA_NTracks.svg")
plt.show()

In [ ]:
# plot SPD tracklets vs z vertex
hist_name = "QA_SPD_MultiplicityVsVertexZ"

hist_dicts = [hist_dict_sig, hist_dict_bkg]

plt.figure(figsize=(len(hist_dicts) * 6, 6))

for i in range(len(hist_dicts)):

    plt.subplot(1, len(hist_dicts), i + 1)

    hist_dict = hist_dicts[i]

    values, xedges, yedges = hist_dict[hist_name].to_numpy()
    xcenters = (xedges[:-1] + xedges[1:]) / 2
    ycenters = (yedges[:-1] + yedges[1:]) / 2
    plt.pcolormesh(xedges, yedges, values.T, cmap='inferno')

    plt.ylabel('SPD # tracklets', loc='top', fontsize=12, fontweight='bold')
    plt.xlabel('SPD z-vertex', loc='right', fontsize=12, fontweight='bold')

    plt.minorticks_on()
    plt.tick_params(direction='out', axis='both', which='both')
    plt.tick_params(which='minor', length=3)
    plt.tick_params(which='major', length=6)
    # plt.tick_params(axis='y', which="both", colors='white')

    plt.colorbar(pad=0.)

plt.tight_layout()
if (save_fig): plt.savefig("QA_SPD_MultiplicityVsVertexZ.svg")
plt.show()

In [ ]:
# plot SPD phi vs SPD eta
hist_name = "QA_SPD_PhiVsEta"

hist_dicts = [hist_dict_sig, hist_dict_bkg]

plt.figure(figsize=(len(hist_dicts) * 6, 6))

for i in range(len(hist_dicts)):

    plt.subplot(1, len(hist_dicts), i + 1)

    hist_dict = hist_dicts[i]

    values, xedges, yedges = hist_dict[hist_name].to_numpy()
    xcenters = (xedges[:-1] + xedges[1:]) / 2
    ycenters = (yedges[:-1] + yedges[1:]) / 2
    weights = values # / np.max(values)
    plt.imshow(weights.T, origin='lower', extent=(xedges[0], xedges[-1], yedges[0], yedges[-1]), aspect='auto', cmap='inferno')

    plt.ylabel(r'SPD $\phi$ [rad]', loc='top', fontsize=12, fontweight='bold')
    plt.xlabel(r'SPD $\eta$', loc='right', fontsize=12, fontweight='bold')

    plt.minorticks_on()
    plt.tick_params(direction='out', axis='both', which='both')
    plt.tick_params(which='minor', length=3)
    plt.tick_params(which='major', length=6)
    # plt.tick_params(axis='y', which="both", colors='white')

    plt.colorbar(pad=0.)

plt.tight_layout()
if (save_fig): plt.savefig("QA_SPD_PhiVsEta.svg")
plt.show()

In [ ]:
# plot ITS Layer vs Phi
hist_name = "QA_ITS_LayerNoVsPhi"

hist_dicts = [hist_dict_sig, hist_dict_bkg]

plt.figure(figsize=((len(hist_dicts) + 1) * 6, 6))

for i in range(len(hist_dicts)):

    plt.subplot(1, len(hist_dicts) + 1, i + 1)

    hist_dict = hist_dicts[i]

    values, xedges, yedges = hist_dict[hist_name].to_numpy()
    xcenters = (xedges[:-1] + xedges[1:]) / 2
    ycenters = (yedges[:-1] + yedges[1:]) / 2
    plt.pcolormesh(xedges, yedges, values.T, cmap='inferno')

    plt.ylabel('ITS Layer', loc='top', fontsize=12, fontweight='bold')
    plt.xlabel(r'$\phi$', loc='right', fontsize=12, fontweight='bold')

    plt.minorticks_on()
    plt.tick_params(direction='out', axis='both', which='both')
    plt.tick_params(which='minor', length=3)
    plt.tick_params(which='major', length=6)

    plt.colorbar(pad=0.)

# make a third plot, but of the ratio of both prev. plots
plt.subplot(1, len(hist_dicts) + 1, len(hist_dicts) + 1)

values_sig, xedges, yedges = hist_dict_sig[hist_name].to_numpy()
values_bkg, _, _ = hist_dict_bkg[hist_name].to_numpy()
values = values_bkg/ values_sig
plt.pcolormesh(xedges, yedges, values.T, cmap='inferno')

plt.ylabel('ITS Layer', loc='top', fontsize=12, fontweight='bold')
plt.xlabel(r'$\phi$', loc='right', fontsize=12, fontweight='bold')

plt.minorticks_on()
plt.tick_params(direction='out', axis='both', which='both')
plt.tick_params(which='minor', length=3)
plt.tick_params(which='major', length=6)

plt.colorbar(label='Ratio', pad=0.)

# the end
plt.tight_layout()
if (save_fig): plt.savefig("QA_ITS_LayerNoVsPhi.svg")
plt.show()

In [ ]:
# plot ITS layer vs Eta
hist_name = "QA_ITS_LayerNoVsEta"

hist_dicts = [hist_dict_sig, hist_dict_bkg]

plt.figure(figsize=((len(hist_dicts) + 1) * 6, 6))

for i in range(len(hist_dicts)):

    plt.subplot(1, len(hist_dicts) + 1, i + 1)

    hist_dict = hist_dicts[i]

    values, xedges, yedges = hist_dict[hist_name].to_numpy()
    xcenters = (xedges[:-1] + xedges[1:]) / 2
    ycenters = (yedges[:-1] + yedges[1:]) / 2
    plt.pcolormesh(xedges, yedges, values.T, cmap='inferno')

    plt.ylabel('ITS Layer', loc='top', fontsize=12, fontweight='bold')
    plt.xlabel(r'$\eta$', loc='right', fontsize=12, fontweight='bold')

    plt.minorticks_on()
    plt.tick_params(direction='out', axis='both', which='both')
    plt.tick_params(which='minor', length=3)
    plt.tick_params(which='major', length=6)

    plt.colorbar(pad=0.)

# make a third plot, but of the ratio of both prev. plots
plt.subplot(1, len(hist_dicts) + 1, len(hist_dicts) + 1)

values_sig, xedges, yedges = hist_dict_sig[hist_name].to_numpy()
values_bkg, _, _ = hist_dict_bkg[hist_name].to_numpy()
values = values_bkg / values_sig
plt.pcolormesh(xedges, yedges, values.T, cmap='inferno')

plt.ylabel('ITS Layer', loc='top', fontsize=12, fontweight='bold')
plt.xlabel(r'$\eta$', loc='right', fontsize=12, fontweight='bold')

plt.minorticks_on()
plt.tick_params(direction='out', axis='both', which='both')
plt.tick_params(which='minor', length=3)
plt.tick_params(which='major', length=6)

plt.colorbar(label='Ratio', pad=0.)

# the end
plt.tight_layout()
if (save_fig): plt.savefig("QA_ITS_LayerNoVsEta.svg")
plt.show()

In [ ]:
# plot TPC N Sigma [pi,kaon, proton] vs inner param. momentum
for part_name in ["Pion", "Kaon", "Proton"]:

    hist_name = "QA_TPC_NSigma" + part_name + "VsInnerParamP"
    hist_dicts = [hist_dict_sig, hist_dict_bkg]
    hist_label = ["LHC23l1a3", "LHC20e3a"]

    plt.figure(figsize=((len(hist_dicts) + 1) * 6, 6))

    cmap = plt.cm.inferno
    cmap.set_under(color='white')

    for i in range(len(hist_dicts)):

        plt.subplot(1, len(hist_dicts) + 1, i + 1)

        hist_dict = hist_dicts[i]

        values, xedges, yedges = hist_dict[hist_name].to_numpy()
        xcenters = (xedges[:-1] + xedges[1:]) / 2
        ycenters = (yedges[:-1] + yedges[1:]) / 2

        # plt.pcolormesh(xedges, yedges, values.T, cmap=cmap, vmin=1, norm=matplotlib.colors.LogNorm())
        plt.pcolormesh(xedges, yedges, values.T, cmap=cmap, norm="log", label=hist_label[i])

        plt.ylabel(r'TPC $N_{\sigma}$ ' + part_name, loc='top', fontsize=12, fontweight='bold')
        plt.xlabel(r'$p^{in}$ (GeV/c)', loc='right', fontsize=12, fontweight='bold')

        plt.minorticks_on()
        plt.tick_params(direction='in', axis='both', which='both', top=True, right=True)
        plt.tick_params(which='minor', length=3)
        plt.tick_params(which='major', length=6)

        plt.colorbar(pad=0.)

        # add a text box
        plt.text(0.75, 0.925, hist_label[i], fontsize=12, transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.8, boxstyle="round", pad=0.4))

    # make a third plot, but of the ratio of both prev. plots
    plt.subplot(1, len(hist_dicts) + 1, 3)

    values_bkg, xedges, yedges = hist_dict_bkg["QA_TPC_NSigma" + part_name + "VsEta"].to_numpy()
    values_sig, __, __ = hist_dict_sig["QA_TPC_NSigma" + part_name + "VsEta"].to_numpy()

    mean_bkg = []
    std_dev_bkg = []
    for i in range(len(values_bkg)):
        weights = values_bkg[i]
        mean_value = np.sum(weights * yedges[:-1]) / np.sum(weights)
        std_dev_value = np.sqrt(np.sum(weights * (yedges[:-1] - mean_value)**2) / np.sum(weights))
        mean_bkg.append(mean_value)
        std_dev_bkg.append(std_dev_value)

    mean_sig = []
    std_dev_sig = []
    for i in range(len(values_sig)):
        weights = values_sig[i]
        mean_value = np.sum(weights * yedges[:-1]) / np.sum(weights)
        std_dev_value = np.sqrt(np.sum(weights * (yedges[:-1] - mean_value)**2) / np.sum(weights))
        mean_sig.append(mean_value)
        std_dev_sig.append(std_dev_value)

    bin_width_sig = xedges[1] - xedges[0]
    plt.errorbar( (xedges[:-1] + xedges[1:]) / 2, std_dev_bkg, xerr=bin_width_sig/2, fmt='o', label="Std. Dev. LHC20e3a", color=style.colors[1], markersize=4)
    plt.errorbar( (xedges[:-1] + xedges[1:]) / 2, std_dev_sig, xerr=bin_width_sig/2, fmt='o', label="Std. Dev. LHC23l1a3", color=style.colors[3], markersize=4)
    plt.errorbar( (xedges[:-1] + xedges[1:]) / 2, mean_bkg, xerr=bin_width_sig/2, fmt='o', label="Mean LHC20e3a", color=style.colors[0], markersize=4)
    plt.errorbar( (xedges[:-1] + xedges[1:]) / 2, mean_sig, xerr=bin_width_sig/2, fmt='o', label="Mean LHC23l1a3", color=style.colors[2], markersize=4)

    plt.legend(loc="upper center", fontsize=12, borderaxespad=1.)

    plt.xlabel(r'$\eta$', loc='right', fontsize=12, fontweight='bold')

    plt.ylim(top=2.5, bottom=-1)

    plt.minorticks_on()
    plt.tick_params(direction='in', axis='both', which='both', top=True, right=True)
    plt.tick_params(which='minor', length=3)
    plt.tick_params(which='major', length=6)

    # the end
    plt.tight_layout()
    if (save_fig): plt.savefig("QA_TPC_NSigmaVsInnerParamP.svg")
    plt.show()

In [ ]:
# plot ratio TPC N sigma [pion, kaon, proton] vs [inner param. momentum, eta]
part_names = ["Pion", "Kaon", "Proton"]

for part_name in part_names:

    hist_names = ["QA_TPC_NSigma" + part_name + "VsInnerParamP", "QA_TPC_NSigma" + part_name + "VsEta"]
    var_names = [r'$p^{in}$ (GeV/c)', r'$\eta$']

    plt.figure(figsize=((len(hist_names) + 1) * 6, 6))

    for i in range(len(hist_names)):

        plt.subplot(1, len(hist_names) + 1, i + 1)

        hist_name = hist_names[i]

        values_sig, xedges, yedges = hist_dict_sig[hist_name].to_numpy()
        values_bkg, __, __ = hist_dict_bkg[hist_name].to_numpy()
        values = values_bkg / values_sig
        xcenters = (xedges[:-1] + xedges[1:]) / 2
        ycenters = (yedges[:-1] + yedges[1:]) / 2
        plt.pcolormesh(xedges, yedges, values.T, cmap='inferno')

        plt.ylabel(r'TPC $N_{\sigma}$ ' + part_name, loc='top', fontsize=12, fontweight='bold')
        plt.xlabel(var_names[i], loc='right', fontsize=12, fontweight='bold')

        plt.minorticks_on()
        plt.tick_params(direction='out', axis='both', which='both')
        plt.tick_params(which='minor', length=3)
        plt.tick_params(which='major', length=6)

        cbar = plt.colorbar(label='Ratio', pad=0.)
        cbar.set_label('Ratio', rotation=0, fontsize=12, fontweight='bold', loc="top")
        cbar.ax.minorticks_on()
        cbar.ax.tick_params(direction='in', which="both", color='white', width=2)
        cbar.ax.tick_params(which='minor', length=3)
        cbar.ax.tick_params(which='major', length=6)

    plt.tight_layout()
    if (save_fig): plt.savefig("QA_Ratio_TPC_NSigma.svg")
    plt.show()